In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
sample_sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [36]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.Survived.value_counts()/len(train)

In [ ]:
train.Pclass.value_counts()/len(train)

In [ ]:
test.Pclass.value_counts()/len(test)

In [ ]:
train.Embarked.value_counts()/len(train)

In [ ]:
test.Embarked.value_counts()/len(test)

In [ ]:
train.Sex.value_counts()/len(train)

In [ ]:
test.Sex.value_counts()/len(test)

In [ ]:
train[(train.Sex=='female') & (train.Survived==1)].count()/train[(train.Survived==1)].count()

In [ ]:
#Plot Histograms

%matplotlib inline
import matplotlib.pyplot as plt
def plot_hist(ser, bins=20):
    plt.figure(facecolor=(1, 1, 1))
    plt.hist(ser, bins=bins)
    plt.title(ser.name)
    plt.show()
    
plot_hist(train['Age'])
plot_hist(train['Pclass'])
plot_hist(train['SibSp'])
plot_hist(train['Parch'])
plot_hist(train['Fare'])

In [ ]:
plot_hist(test['Age'])

In [72]:
#Categorical and Numerical Attributes

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

num_attr = ['Age', 'Fare', 'SibSp', 'Parch']
cat_attr = ['Sex', 'Embarked', 'Pclass']

#Separate the label
X = train.drop('Survived',  axis=1)
y = train['Survived'].copy()


#Transformer to Impute categorical attr with 'most frequent'

class CatImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.imp_val_ = pd.Series([X[c].value_counts().index[0] for c in X], 
                                 index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.imp_val_)
    
dummy_pipeline = Pipeline([('dummy', None)])

cat_pipeline = Pipeline([
                        ('cat_imputer', CatImputer()),
                        ('ohe', OneHotEncoder())
                        ])

num_pipeline = Pipeline([
                        ('num_imputer', SimpleImputer(strategy='median')),
                        ('scaler', StandardScaler()),
                        ])

feat_pipeline = ColumnTransformer([
                        ('num', num_pipeline, num_attr),
                        ('cat', cat_pipeline, cat_attr)
                        ])

X_train = feat_pipeline.fit_transform(X)

In [ ]:
X_train.shape

In [42]:
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [62]:
s = pd.Series([X[c].value_counts().index[0] for c in X[cat_attr].columns], index=X[cat_attr].columns)
type(X.fillna(s))
# X.info()

pandas.core.frame.DataFrame